In [1]:
import threading

def merge(list1, list2):
    """Merge two sorted lists into a single sorted list."""
    merged = []
    i = j = 0
    while i < len(list1) and j < len(list2):
        if list1[i] < list2[j]:
            merged.append(list1[i])
            i += 1
        else:
            merged.append(list2[j])
            j += 1
    merged += list1[i:] + list2[j:]
    return merged

def sort_worker(input_list, output_list, start, end, thread_semaphore, output_semaphore):
    """Worker function for sorting a portion of the input list."""
    portion = input_list[start:end]
    portion.sort()
    output_semaphore.acquire()
    output_list[start:end] = portion
    output_semaphore.release()
    thread_semaphore.release()

def concurrent_sort(input_list, num_threads):
    """Sort the input list concurrently using the specified number of threads."""
    output_list = [0] * len(input_list)
    portion_size = len(input_list) // num_threads
    thread_semaphores = [threading.Semaphore(0) for _ in range(num_threads)]
    output_semaphore = threading.Semaphore(1)
    threads = []
    for i in range(num_threads):
        start = i * portion_size
        end = start + portion_size if i < num_threads - 1 else len(input_list)
        thread = threading.Thread(target=sort_worker, args=(input_list, output_list, start, end, thread_semaphores[i], output_semaphore))
        threads.append(thread)
        thread.start()

    for semaphore in thread_semaphores:
        semaphore.acquire()

    for i in range(1, num_threads):
        output_list[:i*portion_size] = merge(output_list[:i*portion_size], output_list[i*portion_size:(i+1)*portion_size])

    return output_list
